In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.graph_objects as go


In [ ]:
filename_DD = f"../data/df_DD_2025-04-15_17-01-11.csv"
filename_FB = f"../data/df_DD_2025-04-15_17-01-11.csv"
df_DD = pd.read_csv(filename_DD, parse_dates=["datetime_hour"], index_col=None)
df_FB = pd.read_csv(filename_FB, parse_dates=["datetime_hour"], index_col=None)


In [ ]:
dropped_colnames = [col for col in df_DD.columns if col.startswith("weekday")]
dropped_colnames.extend(["return_count", "event_count_end", "hex_id"])
print(dropped_colnames)
df_DD.drop(columns=dropped_colnames, inplace=True, errors="ignore")

In [ ]:
df_DD.rename(columns={"datetime_hour": "date", "rent_count": "Bike trips", "event_count_start": "Public events"}, inplace=True, errors="ignore")


In [ ]:
df_DD.date = df_DD.date.dt.date

In [ ]:
df_DD.columns

In [ ]:
df = df_DD.groupby("date").agg({"Bike trips": "sum",
                                "Temperature": "mean",
                                "Humidity": "mean", 
                                "Precipitation": "sum",
                                "Wind": "mean",
                                 "is_dayoff": "max",
                                "Public events": "sum" }).reset_index()

In [ ]:
df

In [ ]:
# ## issue: you cannot prevent sns from connecting the lines
# sns.set_style("whitegrid") 
# fix, ax1 = plt.subplots(figsize=(10, 6))
# ax1_name = "Bike trips"
# ax2_name = "Temperature"

# sns.lineplot(data=df, x="date", y=ax1_name, ax=ax1, color="steelblue", label=ax1_name)
# ax2 = ax1.twinx()
# sns.lineplot(data=df, x="date", y="Temperature", ax=ax2, color="firebrick", label=ax2_name)
# ax1.set_ylabel(ax1_name)
# ax2.set_ylabel(ax2_name)
# plt.legend()
# plt.show()

In [ ]:
df.date.iloc[-1]

In [ ]:
whole_range = pd.date_range(start=df.date.iloc[0], end=df.date.iloc[-1], freq="D")
whole_range = whole_range[~whole_range.month.isin([5,6,7,8])]
df = df.set_index("date")
df = df.reindex(whole_range)
df = df.reset_index()
df.rename(columns={"index": "date"}, inplace=True)

In [ ]:
df.sample(10)

In [ ]:
color_dict = {
    "Bike trips": "steelblue",
    "Temperature": "firebrick",
    "Humidity": "darkorange",
    "Precipitation": "mediumseagreen",
    "Wind": "purple",
    "is_dayoff": "black",
    "Public events": "blueviolet"
}

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
df

In [ ]:
df["date_str"] = df.date.dt.strftime("%Y-%m-%d")

In [ ]:
df["month_name"] = df.date.dt.month_name()

In [ ]:
# good plot
fig = go.Figure()
ax1_name = "Bike trips"
ax2_name = "Temperature"

fig.add_trace(go.Scatter( x=df["date_str"], y=df[ax1_name], name=ax1_name, line=dict(color="steelblue"), connectgaps=False, ))

fig.add_trace(go.Scatter( x=df["date_str"], y=df[ax2_name], name=ax2_name, yaxis="y2",line=dict(color=color_dict[ax2_name]),connectgaps=False, ))

# Layout with two y-axes
fig.update_layout( yaxis=dict(title=ax1_name, range=[0,None]),
    yaxis2=dict(title=ax2_name, overlaying="y", side="right" ),
    legend=dict(x=0.5, y=1.1, orientation="h", xanchor="center"),
    width=800, height=600, template="plotly_white", xaxis_type='category' )

fig.show()


In [ ]:
secondary_factors = ["Temperature", "Humidity", "Precipitation", "Wind", "Public events"]

In [ ]:
smaller_fontsize = 9

In [ ]:
whole_range

In [ ]:
# df.date = df.date.astype("str")

In [ ]:
df.date.dt.month.unique()

In [ ]:
df.Temperature.max()

In [ ]:
secondary_y_ranges = {
    "Temperature": [-10, 30],
    "Humidity": [0, 100],
    "Precipitation": [0, 200],
    "Wind": [0, 20],
    "Public events": [0, 250]
}

In [ ]:
df.sort_values("date", inplace=True)

In [ ]:
# TODO make font size smalles everywhere


fig = make_subplots(rows = len(secondary_factors), cols=1, shared_xaxes=True, vertical_spacing=0.02,
                    specs=[[{"secondary_y": True}] for _ in secondary_factors] ) 

ax1_name = "Bike trips"
for i, ax2_name in enumerate(secondary_factors):
    row = i+1

    fig.add_trace(go.Scatter( x=df["date_str"], y=df[ax1_name], name=ax1_name, line=dict(color="steelblue", width=1.5), connectgaps=False, ), row=row, col=1, secondary_y=False )
    fig.add_trace(go.Scatter( x=df["date_str"], y=df[ax2_name], name=ax2_name, yaxis="y2",line=dict(color="darkorange", width=1.5),connectgaps=False, ), row=row, col=1, secondary_y=True )

    fig.update_yaxes(title_text="Bike trips", row=i+1, col=1, secondary_y=False, range=[0, 15000], title_font=dict(size=smaller_fontsize), tickfont=dict(size=smaller_fontsize))
    fig.update_yaxes(title_text=ax2_name, row=i+1, col=1, secondary_y=True, range=secondary_y_ranges[ax2_name], title_font=dict(size=smaller_fontsize), tickfont=dict(size=smaller_fontsize)) 

fig.update_layout(height = 180*len(secondary_factors), width=600, template="plotly_white", showlegend=False, xaxis=dict(tickangle=-90), font=dict(size=smaller_fontsize) )
fig.update_xaxes(title_text="Date", row=len(secondary_factors), col=1, title_font=dict(size=smaller_fontsize), tickfont=dict(size=9), ticklabelstep=11, )
fig.update_yaxes(showgrid=False)
fig.update_xaxes(type='category')
fig.update_xaxes(showgrid=False)
fig.update_xaxes(tickangle=-90)
fig.update_xaxes(ticklabelstep=3)
fig.show()
    